In [628]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from dataset import VNAgriDataset

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

import pickle
import bisect
from statsmodels.tsa.stattools import adfuller

In [629]:
caphe_df = pd.read_csv("../../data/pre_caphe.csv")
luagao_df = pd.read_csv("../../data/pre_luagao.csv")
rauqua_df = pd.read_csv("../../data/pre_rauqua.csv")

In [630]:
nganh_caphe = pd.Series(["Cà phê"] * len(caphe_df), name="Ngành_hàng")
nganh_luagao = pd.Series(["Lua gao"] * len(luagao_df), name="Ngành_hàng")
nganh_rauqua = pd.Series(["Rau quả"] * len(rauqua_df), name="Ngành_hàng")

In [631]:
caphe_df = pd.concat([caphe_df, nganh_caphe], axis=1)
luagao_df = pd.concat([luagao_df, nganh_luagao], axis=1)
rauqua_df = pd.concat([rauqua_df, nganh_rauqua], axis=1)

In [632]:
pre_data = pd.concat([caphe_df, luagao_df, rauqua_df], axis=0, ignore_index=True)

In [633]:
pre_data.to_csv("../../data/pre_data.csv", index=False)

In [634]:
pre_data.shape

(48736, 9)

In [635]:
pre_data.head()

,Tên_mặt_hàng,Thị_trường,Loại_giá,Đơn_vị_tính,Loại_tiền,Nguồn,Ngày,Giá,Ngành_hàng
0,Cà phê Robusta nhân xô,Đắk Lắk,Thương lái thu mua,Vnđ/Kg,VNĐ,CTV địa phương,2025-05-09,128233.0,Cà phê
1,Cà phê Robusta nhân xô,Đắk Nông,Thương lái thu mua,Vnđ/Kg,VNĐ,CTV địa phương,2025-05-09,128350.0,Cà phê
2,Cà phê Robusta nhân xô,Gia Lai,Thương lái thu mua,Vnđ/Kg,VNĐ,CTV địa phương,2025-05-09,128233.0,Cà phê
3,Cà phê Robusta nhân xô,Kon Tum,Thương lái thu mua,Vnđ/Kg,VNĐ,CTV địa phương,2025-05-09,128200.0,Cà phê
4,Cà phê Robusta nhân xô,Lâm Đồng,Thương lái thu mua,Vnđ/Kg,VNĐ,CTV địa phương,2025-05-09,128000.0,Cà phê


In [636]:
pre_data["Ngày"] = pd.to_datetime(pre_data["Ngày"], format="%Y-%m-%d")

# Kiểm tra cột tên mặt hàng

In [637]:
pre_data["Tên_mặt_hàng"].unique()[:5]

array(['Cà phê Robusta nhân xô', 'Cà phê Arabica nhân xô', 'CLC 4900',
       'Jasmine', 'OM 18'], dtype=object)

In [638]:
print(len(pre_data["Tên_mặt_hàng"].unique()))

394


## Xoá các mặt hàng có số lượng record ít

In [639]:
items = pre_data["Tên_mặt_hàng"].unique()
n_items = []

for item in items:
    n = len(pre_data[pre_data["Tên_mặt_hàng"] == item])
    n_items.append(n)

In [640]:
q1 = np.quantile(n_items, 0.5)

In [641]:
pre_data = pre_data.groupby("Tên_mặt_hàng").filter(lambda x: len(x) >= q1)

In [642]:
len(pre_data["Tên_mặt_hàng"].unique())

197

In [643]:
q1

np.float64(59.5)

# Kiểm tra cột thị trường

In [644]:
pre_data["Thị_trường"].unique()

array(['Đắk Lắk', 'Đắk Nông', 'Gia Lai', 'Kon Tum', 'Lâm Đồng',
       'Hồ Chí Minh', 'Cần Thơ', 'Sóc Trăng', 'Đồng Tháp', 'Kiên Giang',
       'An Giang', 'Tiền Giang', 'Trà Vinh', 'Hậu Giang', 'Thái Bình',
       'Hà Nội', 'Bến Tre', 'Cà Mau', 'Long An', 'Vĩnh Long', 'Bạc Liêu',
       'Sơn La'], dtype=object)

In [645]:
print(len(pre_data["Thị_trường"].unique()))

22


# Kiểm tra cột loại giá

In [646]:
pre_data["Loại_giá"].unique()

array(['Thương lái thu mua', 'Công ty thu mua', 'Đại lý thu mua',
       'Thu mua', 'Bán ra', 'Xuất khẩu', 'Tại chợ', 'Bán lẻ', 'Khác',
       'Thu mua tại vườn', 'Bán buôn', 'Vựa thu mua'], dtype=object)

In [647]:
print(len(pre_data["Loại_giá"].unique()))

12


# Kiểm tra đơn vị tính

In [648]:
pre_data["Đơn_vị_tính"].unique()

array(['Vnđ/Kg', 'VNĐ/kg', 'VNĐ/Kg', 'Đồng/kg', 'VNĐ/quả', 'VNĐ/mớ',
       'VNĐ/Chục quả', 'VNĐ/củ', 'VNĐ/Quả', 'VNĐ/cây'], dtype=object)

In [649]:
print(len(pre_data["Đơn_vị_tính"].unique()))

10


In [650]:
pre_data["Đơn_vị_tính"] = pre_data["Đơn_vị_tính"].apply(
    lambda x: "Đồng/kg" if x in [
        'Vnđ/Kg', 'VNĐ/kg', 'VNĐ/Kg', 'Đồng/kg', 'VNĐ/củ', 'VNĐ/Quả'] else x
)

In [651]:
pre_data["Đơn_vị_tính"].unique()

array(['Đồng/kg', 'VNĐ/quả', 'VNĐ/mớ', 'VNĐ/Chục quả', 'VNĐ/cây'],
      dtype=object)

# Kiểm tra cột loại tiền

In [652]:
pre_data["Loại_tiền"].unique()

array(['VNĐ'], dtype=object)

In [653]:
del pre_data["Loại_tiền"]

In [654]:
pre_data.head()

,Tên_mặt_hàng,Thị_trường,Loại_giá,Đơn_vị_tính,Nguồn,Ngày,Giá,Ngành_hàng
0,Cà phê Robusta nhân xô,Đắk Lắk,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128233.0,Cà phê
1,Cà phê Robusta nhân xô,Đắk Nông,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128350.0,Cà phê
2,Cà phê Robusta nhân xô,Gia Lai,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128233.0,Cà phê
3,Cà phê Robusta nhân xô,Kon Tum,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128200.0,Cà phê
4,Cà phê Robusta nhân xô,Lâm Đồng,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-09,128000.0,Cà phê


# Kiểm tra giá trị ADF

In [655]:
adf_columns = [
    "adf_stats", 
    "n_lags", 
    "p_value", 
    "1%", 
    "5%", 
    "10%", 
    "Tên_mặt_hàng"
]
adf_results = pd.DataFrame(columns=adf_columns)

items = pre_data["Tên_mặt_hàng"].unique()
for item in items:

    try:
        result = adfuller(pre_data[pre_data["Tên_mặt_hàng"]==item]["Giá"])
    except ValueError as e:
        continue
    
    temp_df = pd.DataFrame({
        "adf_stats": result[0], 
        "p_value": result[1], 
        "n_lags": result[2], 
        "1%": result[4]["1%"], 
        "5%": result[4]["5%"], 
        "10%": result[4]["10%"], 
        "Tên_mặt_hàng": item
    }, index=[0])

    adf_results = pd.concat([adf_results, temp_df])
    
adf_results.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_5992\3169476205.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  adf_results = pd.concat([adf_results, temp_df])
d:\Apps\Miniconda\envs\project3\Lib\site-packages\statsmodels\regression\linear_model.py:955: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


,adf_stats,n_lags,p_value,1%,5%,10%,Tên_mặt_hàng
0,-1.544740,31,0.511275,-3.431717,-2.862144,-2.567092,Cà phê Robusta nhân xô
0,-0.957852,6,0.768325,-3.491818,-2.888444,-2.581120,CLC 4900
0,-1.002442,17,0.752373,-3.451215,-2.870730,-2.571666,Jasmine
0,-2.137725,12,0.229618,-3.450022,-2.870207,-2.571387,OM 18
0,-3.004943,9,0.034435,-3.461136,-2.875079,-2.573986,OM 5451


In [656]:
success_adf_test = adf_results[
    (adf_results["adf_stats"] < adf_results["10%"]) & 
    (adf_results["p_value"] < 0.05)
]

In [657]:
len(success_adf_test)

65

In [658]:
filter_items = success_adf_test["Tên_mặt_hàng"].unique()

In [659]:
filtered_data = pre_data[pre_data["Tên_mặt_hàng"].isin(filter_items)]

In [660]:
print(len(filtered_data))

17333


In [661]:
filtered_data.head()

,Tên_mặt_hàng,Thị_trường,Loại_giá,Đơn_vị_tính,Nguồn,Ngày,Giá,Ngành_hàng
4842,OM 5451,Cần Thơ,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-19,6000.0,Lua gao
4847,OM 5451,Sóc Trăng,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-19,7125.0,Lua gao
4856,Gạo NL 25% tấm,Kiên Giang,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-16,8460.0,Lua gao
4860,Gạo XK 5% tấm,Kiên Giang,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-16,10070.0,Lua gao
4865,OM 5451,Kiên Giang,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-16,5900.0,Lua gao


# Lập metadata trước khi được chuẩn hoá thời gian và Label Encoder

# Chuẩn hoá mốc thời gian

## Thêm 1 cột ngày được chuyển sang kiểu int cho dễ

In [662]:
filtered_data.columns

Index(['Tên_mặt_hàng', 'Thị_trường', 'Loại_giá', 'Đơn_vị_tính', 'Nguồn',
       'Ngày', 'Giá', 'Ngành_hàng'],
      dtype='object')

In [663]:
filtered_data.head()

,Tên_mặt_hàng,Thị_trường,Loại_giá,Đơn_vị_tính,Nguồn,Ngày,Giá,Ngành_hàng
4842,OM 5451,Cần Thơ,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-19,6000.0,Lua gao
4847,OM 5451,Sóc Trăng,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-19,7125.0,Lua gao
4856,Gạo NL 25% tấm,Kiên Giang,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-16,8460.0,Lua gao
4860,Gạo XK 5% tấm,Kiên Giang,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-16,10070.0,Lua gao
4865,OM 5451,Kiên Giang,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-16,5900.0,Lua gao


In [664]:
items = filtered_data["Tên_mặt_hàng"].unique()

In [665]:
items[:5]

array(['OM 5451', 'Gạo NL 25% tấm', 'Gạo XK 5% tấm', 'Lúa IR 50404',
       'Lúa Jasmine'], dtype=object)

In [666]:
from datetime import datetime

start_date = filtered_data["Ngày"].min()
end_date = filtered_data["Ngày"].max()

In [667]:
filtered_data["Ngày(int)"] = (filtered_data["Ngày"] - start_date).dt.days

C:\Users\DELL\AppData\Local\Temp\ipykernel_5992\827847078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["Ngày(int)"] = (filtered_data["Ngày"] - start_date).dt.days


In [668]:
filtered_data.head()

,Tên_mặt_hàng,Thị_trường,Loại_giá,Đơn_vị_tính,Nguồn,Ngày,Giá,Ngành_hàng,Ngày(int)
4842,OM 5451,Cần Thơ,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-19,6000.0,Lua gao,1596
4847,OM 5451,Sóc Trăng,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-19,7125.0,Lua gao,1596
4856,Gạo NL 25% tấm,Kiên Giang,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-16,8460.0,Lua gao,1593
4860,Gạo XK 5% tấm,Kiên Giang,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-16,10070.0,Lua gao,1593
4865,OM 5451,Kiên Giang,Thương lái thu mua,Đồng/kg,CTV địa phương,2025-05-16,5900.0,Lua gao,1593


In [669]:
weekly_index = pd.date_range(
    start=start_date,
    end=end_date,
    freq="W"
)

In [670]:
weekly_index = (weekly_index - start_date).days

## Sử dụng nội suy tuyến tinh để tính các điểm dữ liệu

In [ ]:
import pandas as pd
import numpy as np
import bisect

# Chuẩn bị danh sách để chứa kết quả
interpolated_rows = []

for item in items:
    item_df = filtered_data[filtered_data["Tên_mặt_hàng"] == item]

    # Lấy giá trị ngày dưới dạng int (số ngày từ start_date)
    date_int = item_df["Ngày(int)"]
    
    min_item_date = np.min(date_int)
    max_item_date = np.max(date_int)

    # Tìm chỉ số trong weekly_index tương ứng
    min_week_range = bisect.bisect_left(weekly_index, min_item_date)
    max_week_range = bisect.bisect_right(weekly_index, max_item_date)

    # Các mốc tuần cần lấy
    week_item_range = weekly_index[min_week_range: max_week_range]

    # Nội suy giá trị
    item_interp = np.interp(week_item_range, date_int, item_df["Giá"])

    # Với mỗi tuần, tạo một dòng mới giữ lại các thuộc tính khác (dùng mode hoặc first)
    for week_day, price in zip(week_item_range, item_interp):
        row = {
            "Tên_mặt_hàng": item,
            "Ngày(int)": week_day,
            "Giá": price
        }
        # Thêm các cột khác nếu có
        for col in ['Thị_trường', 'Loại_giá', 'Đơn_vị_tính', 'Nguồn', 'Ngành_hàng']:
            if col in item_df.columns:
                row[col] = item_df[col].iloc[0]
        interpolated_rows.append(row)

# Gộp lại thành DataFrame
interpolated_df = pd.DataFrame(interpolated_rows)

# Nếu muốn, có thể chuyển lại "Ngày(int)" → "Ngày"
interpolated_df["Ngày"] = pd.to_datetime(start_date) + pd.to_timedelta(interpolated_df["Ngày(int)"], unit="D")


In [672]:
len(interpolated_df["Ngày(int)"].unique())

228

In [673]:
interpolated_df

,Tên_mặt_hàng,Ngày(int),Giá,Ngày
0,OM 5451,1147,8350.0,2024-02-25
1,OM 5451,1154,8350.0,2024-03-03
2,OM 5451,1161,8350.0,2024-03-10
3,OM 5451,1168,8350.0,2024-03-17
4,OM 5451,1175,8350.0,2024-03-24
...,...,...,...,...
10241,Đu đủ ruột đỏ,300,15000.0,2021-10-31
10242,Đu đủ ruột đỏ,307,15000.0,2021-11-07
10243,Đu đủ ruột đỏ,314,15000.0,2021-11-14
10244,Đu đủ ruột đỏ,321,15000.0,2021-11-21


In [676]:
interpolated_df.to_csv("../../data/interpolated.csv", index=False)